# Financial Analyst

In [3]:
import os
import json
import requests
from openai import OpenAI
import time
from dotenv import load_dotenv, find_dotenv


In [4]:
_:bool = load_dotenv(find_dotenv())
client:OpenAI = OpenAI()
FMP_API_KEY = os.environ["FMP_API_KEY"]


# defining functions

In [5]:
def get_income_statement(ticker,period):
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?period={period}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_balance_sheet(ticker,period):
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?period={period}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_cash_flow_statement(ticker,period):
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?period={period}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_key_metrics(ticker,period):
    url = f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?period={period}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_financial_growth(ticker,period):
    url = f"https://financialmodelingprep.com/api/v3/financial-growth/{ticker}?period={period}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_financial_ratios(ticker,period):
    url = f"https://financialmodelingprep.com/api/v3/ratios/{ticker}?period={period}&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())  



In [6]:
available_functions = {
    "get_income_statement":get_income_statement,
    "get_balance_sheet":get_balance_sheet,
    "get_cash_flow_statement":get_cash_flow_statement,
    "get_key_metrics":get_key_metrics,
    "get_financial_growth":get_financial_growth,
    "get_financial_ratios":get_financial_ratios
}

In [7]:
def run_asisstant(prompt):
    assistant = client.beta.assistants.create(
        name = "financial analyst",
        model = "gpt-3.5-turbo-1106",
        instructions="Act as a financial analyst by accessing financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends",
        tools=[{
    "type":"function",
    "function":{
        "name":"get_income_statement",
        "description":"you get the income statement",
        "parameters":{
            "type":"object",
            "properties":{
                "ticker":{
                    "type":"string"
                },
                "period":{
                    "type":"string"
                }
            }
        }
    }
},
{
    "type":"function",
    "function":{
        "name":"get_balance_sheet",
        "description":"you get the balance sheet",
        "parameters":{
            "type":"object",
            "properties":{
                "ticker":{
                    "type":"string"
                },
                "period":{
                    "type":"string"
                }
            }
        }
    }
},
{
    "type":"function",
    "function":{
        "name":"get_cash_flow_statement",
        "description":"you get the cash flow statement",
        "parameters":{
            "type":"object",
            "properties":{
                "ticker":{
                    "type":"string"
                },
                "period":{
                    "type":"string"
                }
            }
        }
    }
},
{
    "type":"function",
    "function":{
        "name":"get_key_metrics",
        "description":"you get the key metrics",
        "parameters":{
            "type":"object",
            "properties":{
                "ticker":{
                    "type":"string"
                },
                "period":{
                    "type":"string"
                }
            }
        }
    }
},
{
    "type":"function",
    "function":{
        "name":"get_financial_growth",
        "description":"you get the financial growth",
        "parameters":{
            "type":"object",
            "properties":{
                "ticker":{
                    "type":"string"
                },
                "period":{
                    "type":"string"
                }
            }
        }
    }
},
{
    "type":"function",
    "function":{
        "name":"get_financial_ratios",
        "description":"you get the financial ratios",
        "parameters":{
            "type":"object",
            "properties":{
                "ticker":{
                    "type":"string"
                },
                "period":{
                    "type":"string"
                }
            }
        }
    }
},
{"type":"code_interpreter"}
]
    )
    thread = client.beta.threads.create()
    message = client.beta.threads.messages.create(
        thread_id = thread.id,
        role="user",
        content=prompt,
        
    )
    return assistant,message,thread
    

In [18]:
assistant,message,thread = run_asisstant("how do i manage my budget with 5000rs a month")


In [19]:
assistant,message,thread = run_asisstant("from the budget remove 2000rs")


In [20]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [21]:
while True:
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,  
        run_id=run.id
    )
    run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
    if run.status == "requires_action":
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        tool_outputs = []

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_arguments = json.loads(tool_call.function.arguments)

            if function_name in available_functions:
                function_to_call = available_functions[function_name]
                function_response = function_to_call(
                    ticker=function_arguments["ticker"],
                    period=function_arguments["period"],
                )
                tool_outputs.append({
                    "tool_call_id":tool_call.id,
                    "output":function_response
                })
        client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
    elif run.status == "in_progress" or run.status == "queued":
        time.sleep(5)
    elif run.status == "completed":
        answer = client.beta.threads.messages.list(
            thread_id=thread.id
        )
        for ans in reversed(answer.data):
            print(f"{ans.role} : {ans.content}")
        break
    elif run.status == "failed":
        print("failed")
        break
    else:
        print("the answer is unknown")
        break

user : [MessageContentText(text=Text(annotations=[], value='from the budget remove 2000rs'), type='text')]
assistant : [MessageContentText(text=Text(annotations=[], value='The financial data for Apple Inc. (AAPL) for the past five years has been retrieved. Here are some of the key metrics and growth figures for your review:\n\n### Key Metrics:\n- Revenue per Share: $24.34\n- Net Income per Share: $6.16\n- Operating Cash Flow per Share: $7.02\n- Free Cash Flow per Share: $6.33\n- Market Cap: $2.7 Trillion\n- PE Ratio: 27.79\n- Debt to Equity Ratio: 1.79\n- Current Ratio: 0.99\n- Dividend Yield: 0.56%\n- Return on Equity (ROE): 1.56%\n- Book Value per Share: $3.95\n- Tangible Book Value per Share: $3.95\n\n### Financial Growth (2023-2019):\n- Revenue Growth: -2.80%\n- Gross Profit Growth: -0.96%\n- Net Income Growth: -2.81%\n- Operating Cash Flow Growth: -9.50%\n- Free Cash Flow Growth: -10.64%\n- Debt Growth: -7.48%\n- Research and Development Expense Growth: 13.96%\n\nBased on this dat